In [ ]:
from seisbench.data import WaveformDataset

In [ ]:
dataset1 = WaveformDataset(r'X:\DataSets-Local\Ahar-Dataset_2025-04-19T10-37-48',
                           component_order="ZNE")
dataset1.metadata['station_network_code'] = 'AH'
dataset1.metadata['network'] = 'Ahar'

In [ ]:
dataset2 = WaveformDataset(r'X:\DataSets-Local\Ilam-Dataset_2025-04-13T12-42-42',
                           component_order="ZNE")
dataset2.metadata['station_network_code'] = 'IL'
dataset2.metadata['network'] = 'Ilam'

In [ ]:
dataset3 = WaveformDataset(r'X:\DataSets-Local\Kaki-Dataset_2025-04-23T11-38-30',
                           component_order="ZNE")
dataset3.metadata['station_network_code'] = 'KK'
dataset3.metadata['network'] = 'Kaki'

In [ ]:
dataset4 = WaveformDataset(r'X:\DataSets-Local\Qeshm-Dataset_2025-04-12T14-50-37',
                           component_order="ZNE")
dataset4.metadata['station_network_code'] = 'QM'
dataset4.metadata['network'] = 'Qeshm'

In [ ]:
from seisbench.data import MultiWaveformDataset

In [ ]:
combined_dataset = MultiWaveformDataset([dataset1, dataset2, dataset3, dataset4])

In [ ]:
list(combined_dataset.metadata.keys())

In [ ]:
# duplicates = combined_dataset.metadata['trace_name'].duplicated()
# combined_dataset.metadata[duplicates].head()


In [ ]:
from seisbench.data import WaveformDataWriter
from pathlib import Path

In [ ]:
base_path = Path(r'X:\DataSets-Local\Merged_All_DataSets_2025-04-23 (Ahar-Ilam-Kaki-Qeshm) 2')
metadata_path = base_path / "metadata.csv"
waveforms_path = base_path / "waveforms.hdf5"

In [ ]:
with WaveformDataWriter(metadata_path=metadata_path,
                        waveforms_path=waveforms_path) as writer:
    for idx in range(len(combined_dataset)):
        waveform, metadata = combined_dataset.get_sample(idx)
        del metadata['trace_name']
        del metadata['index']
        writer.add_trace(metadata=metadata, waveform=waveform)